In [39]:
import spacy #nlp
import pdfminer #pdf2tx
import re #regex
import os #file manip
import pandas as pd #csv - tabular

In [40]:
import pdf2txt

In [ ]:
# converting pdf to txt

In [44]:
def convert_pdf(f):

    output_filename = os.path.basename(os.path.splitext(f)[0]) + ".txt"
    output_filepath = os.path.join("output/txt/", output_filename)
    pdf2txt.main(args=[f, "--outfile", output_filepath]) #pdf to txt and save it in the given location
    print(output_filepath + " saved successfully!!!")
    return open(output_filepath).read()

In [ ]:
# load the language model

In [5]:
nlp = spacy.load("en_core_web_sm")

In [6]:
#phone_num credit https://stackoverflow.com/a/3868861

In [45]:
result_dict = {'name': [], 'phone': [], 'email': [], 'skills': []} 
names = []
phones = []
emails = []
skills = []

In [46]:
def parse_content(text):
    skillset = re.compile("python|java|sql|hadoop|tableau")
    phone_num = re.compile(
        "(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})"
    )
    doc = nlp(text)
    name = [entity.text for entity in doc.ents if entity.label_ is "PERSON"][0]
    print(name)
    email = [word for word in doc if word.like_email == True][0]
    print(email)
    phone = str(re.findall(phone_num, text.lower()))
    skills_list = re.findall(skillset, text.lower())
    unique_skills_list = str(set(skills_list))
    names.append(name)
    emails.append(email)
    phones.append(phone)
    skills.append(unique_skills_list)
    print("Extraction completed successfully!!!")

In [47]:
for file in os.listdir('resumes/'):
    if file.endswith('.pdf'):
        print('Reading.....' + file)
        txt = convert_pdf(os.path.join('resumes/',file))
        parse_content(txt)

Reading.....Alisson ParkerCV.pdf
output/txt/Alisson ParkerCV.txt saved successfully!!!
Alisson Parker-Wright
alli1414parks@mail.com
Extraction completed successfully!!!
Reading.....John DominicCV.pdf
output/txt/John DominicCV.txt saved successfully!!!
John Dominic 


johndominic@mail.com
Extraction completed successfully!!!
Reading.....AshleyMilesCV.pdf
output/txt/AshleyMilesCV.txt saved successfully!!!
Ashley Miles
ashleymiles@memail.com
Extraction completed successfully!!!


In [52]:
result_dict['name'] = names
result_dict['phone'] = phones
result_dict['email'] = emails
result_dict['skills'] = skills
#print(result_dict)

In [55]:
result_df = pd.DataFrame(result_dict)
result_df

,name,phone,email,skills
0,Alisson Parker-Wright,['8569878511'],alli1414parks@mail.com,"{'python', 'tableau', 'java'}"
1,John Dominic \n\n,['7877756411'],johndominic@mail.com,"{'hadoop', 'python', 'java'}"
2,Ashley Miles,['6592251422'],ashleymiles@memail.com,"{'sql', 'tableau'}"


In [56]:
result_df.to_csv('output/csv/parsed_resumes.csv')